In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import sys
import os.path
sys.path.append('/scratch/pramodkm/vqa/vqa_kazemi2017show/')
import math
import json

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from tqdm import tqdm

import config
import data
import model
import utils

In [3]:
# Load pretrained model
log = torch.load('/scratch/pramodkm/vqa/vqa_kazemi2017show/logs/2017-08-04_00.55.19.pth')
tokens = len(log['vocab']['question']) + 1

net = torch.nn.DataParallel(model.Net(tokens))
net.load_state_dict(log['weights'])

In [4]:
# Load validation data
val_loader = data.get_loader(val=True)
LOADER=val_loader
PREFIX="val"

val E000:   0% 0/950 [00:00<?, ?it/s]

In [13]:
# Iterate over batches (batch_size = 128) and compute accuracy
tq = tqdm(LOADER, desc='{} E{:03d}'.format(PREFIX, 0), ncols=0)
net.eval()
answ = []
idxs = []
accs = []
log_softmax = nn.LogSoftmax().cuda()
num_batches = 0
for v, q, a, idx, q_len in tq:
        var_params = {
            'volatile': True,
            'requires_grad': False,
        }
        v = Variable(v.cuda(async=True), **var_params)
        q = Variable(q.cuda(async=True), **var_params)
        a = Variable(a.cuda(async=True), **var_params)
        q_len = Variable(q_len.cuda(async=True), **var_params)

        out = net(v, q, q_len)
        nll = -log_softmax(out)
        loss = (nll * a / 10).sum(dim=1).mean()
        acc = utils.batch_accuracy(out.data, a.data).cpu()
        
        _, answer = out.data.cpu().max(dim=1)
        answ.append(answer.view(-1))
        accs.append(acc.view(-1))
        idxs.append(idx.view(-1).clone())
        #print(acc.mean())
        num_batches += 1
        if num_batches == 10:
            break

answ = list(torch.cat(answ, dim=0))
accs = list(torch.cat(accs, dim=0))
idxs = list(torch.cat(idxs, dim=0))


val E000:   0% 0/950 [00:00<?, ?it/s]
/scratch/pramodkm/vqa/vqa_kazemi2017show/model.py:135: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention = F.softmax(attention)
/scratch/pramodkm/tensorflow_gpu_python3/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
val E000:   1% 9/950 [00:02<03:42,  4.23it/s]

In [14]:
with open(config.vocabulary_path, 'r') as fd:
    vocab_json = json.load(fd)